In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sentinel-2 Cloud Masking with s2cloudless

Author: jdbcode


This tutorial is an introduction to masking clouds and cloud shadows in Sentinel-2 (S2) surface reflectance (SR) data using Earth Engine. Clouds are identified from the S2 cloud probability dataset (s2cloudless) and shadows are defined by cloud projection intersection with low-reflectance near-infrared (NIR) pixels.

### Run me first

Run the following cell to initialize the Earth Engine API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [2]:
import ee
import pandas as pd
import json

# # Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Assemble cloud mask components

This section builds an S2 SR collection and defines functions to add cloud and cloud shadow component layers to each image in the collection.



### Define collection filter and cloud mask parameters

Define parameters that are used to filter the S2 image collection and determine cloud and cloud shadow identification.

|Parameter | Type | Description |
| :-- | :-- | :-- |
| `AOI` | `ee.Geometry` | Area of interest |
| `START_DATE` | string | Image collection start date (inclusive) |
| `END_DATE` | string | Image collection end date (exclusive) |
| `CLOUD_FILTER` | integer | Maximum image cloud cover percent allowed in image collection |
| `CLD_PRB_THRESH` | integer | Cloud probability (%); values greater than are considered cloud |
| `NIR_DRK_THRESH` | float | Near-infrared reflectance; values less than are considered potential cloud shadow |
| `CLD_PRJ_DIST` | float | Maximum distance (km) to search for cloud shadows from cloud edges |
| `BUFFER` | integer | Distance (m) to dilate the edge of cloud-identified objects |

The values currently set for `AOI`, `START_DATE`, `END_DATE`, and `CLOUD_FILTER` are intended to build a collection for a single S2 overpass of a region near Portland, Oregon, USA. When parameterizing and evaluating cloud masks for a new area, it is good practice to identify a single overpass date and limit the regional extent to minimize processing requirements. If you want to work with a different example, use this [Earth Engine App](https://showcase.earthengine.app/view/s2-sr-browser-s2cloudless-nb) to identify an image that includes some clouds, then replace the relevant parameter values below with those provided in the app.

In [4]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

### CHANGE BELOW PATH ###
zhenya_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david_path = 'D:/canopy_data/csvs/polygons_101320.csv'
polygons = import_aois(zhenya_path)

feature_id = 0 
features = []
for poly in polygons:
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)

fc = ee.FeatureCollection(features)

In [5]:
bad_polygons = [polygons[24], polygons[28], polygons[35], polygons[40], polygons[42], polygons[52], polygons[63], polygons[87], polygons[88], polygons[91], polygons[97]]

In [7]:
Full_Basin = [[[5.493164,8.276727],[5.449219,-5.703448],[31.376953,-4.959615],[31.157227,8.711359],[5.493164,8.276727]]]

AOI = ee.Geometry.Polygon(bad_polygons[0])
AOI_full_basin = ee.Geometry.Polygon(Full_Basin)
START_DATE = '2019-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

### Build a Sentinel-2 collection

[Sentinel-2 surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) and [Sentinel-2 cloud probability](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY) are two different image collections. Each collection must be filtered similarly (e.g., by date and bounds) and then the two filtered collections must be joined.

Define a function to filter the SR and s2cloudless collections according to area of interest and date parameters, then join them on the `system:index` property. The result is a copy of the SR collection where each image has a new `'s2cloudless'` property whose value is the corresponding s2cloudless image.

In [6]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

Apply the `get_s2_sr_cld_col` function to build a collection according to the parameters defined above.

In [9]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

### Define cloud mask component functions

#### Cloud components

Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.

In [16]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

#### Cloud shadow components

Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. Note that the image input needs to be the result of the above `add_cloud_bands` function because it relies on knowing which pixels are considered cloudy (`'clouds'` band).

In [17]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

#### Final cloud-shadow mask

Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.

In [13]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

## Visualize and evaluate cloud mask components

This section provides functions for displaying the cloud and cloud shadow components. In most cases, adding all components to images and viewing them is unnecessary. This section is included to illustrate how the cloud/cloud shadow mask is developed and demonstrate how to test and evaluate various parameters, which is helpful when defining masking variables for an unfamiliar region or time of year.

In applications outside of this tutorial, if you prefer to include only the final cloud/cloud shadow mask along with the original image bands, replace:

```
return img_cloud_shadow.addBands(is_cld_shdw)
```

with

```
return img.addBands(is_cld_shdw)
```

in the above `add_cld_shdw_mask` function.

### Define functions to display image and mask component layers.

Folium will be used to display map layers. Import `folium` and define a method to display Earth Engine image tiles.

In [20]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

Define a function to display all of the cloud and cloud shadow components to an interactive Folium map. The input is an image collection where each image is the result of the `add_cld_shdw_mask` function defined previously.

In [14]:
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12, height=700)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

### Display mask component layers

Map the `add_cld_shdw_mask` function over the collection to add mask component bands to each image, then display the results.

Give the system some time to render everything, it should take less than a minute.

In [15]:
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)

display_cloud_layers(s2_sr_cld_col_eval_disp)

### Evaluate mask component layers

In the above map, use the layer control panel in the upper right corner to toggle layers on and off; layer names are the same as band names, for easy code referral. Note that the layers have a minimum zoom level of 9 to avoid resource issues that can occur when visualizing layers that depend on the `ee.Image.reproject` function (used during cloud shadow project and mask dilation).

Try changing the above `CLD_PRB_THRESH`, `NIR_DRK_THRESH`, `CLD_PRJ_DIST`, and `BUFFER` input variables and rerunning the previous cell to see how the results change. Find a good set of values for a given overpass and then try the procedure with a new overpass with different cloud conditions (this [S2 SR image browser app](https://showcase.earthengine.app/view/s2-sr-browser-s2cloudless-nb) is handy for quickly identifying images and determining image collection filter criteria). Try to identify a set of parameter values that balances cloud/cloud shadow commission and omission error for a range of cloud types. In the next section, we'll use the values to actually apply the mask to generate a cloud-free composite for 2020.

## Apply cloud and cloud shadow mask

In this section we'll generate a cloud-free composite for the same region as above that represents mean reflectance for July and August, 2020.

### Define collection filter and cloud mask parameters

We'll redefine the parameters to be a little more aggressive, i.e. decrease the cloud probability threshold, increase the cloud projection distance, and increase the buffer. These changes will increase cloud commission error (mask out some clear pixels), but since we will be compositing images from three months, there should be plenty of observations to complete the mosaic.

In [7]:
AOI = ee.Geometry.Polygon(bad_polygons[0])
START_DATE = '2019-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

### Build a Sentinel-2 collection

Reassemble the S2-cloudless collection since the collection filter parameters have changed.

In [8]:
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

### Define cloud mask application function

Define a function to apply the cloud mask to each image in the collection.

In [18]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

In [21]:
def apply_cld_shdw_mask_all_bands(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

s2_sr_median_2 = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask_all_bands)
                             .median())


# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12, height=700)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median_2,
                {'bands': ['TCI_R', 'TCI_G', 'TCI_B'], 'min': 0, 'max': 3000, 'gamma': 4},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


### Process the collection

Add cloud and cloud shadow component bands to each image and then apply the mask to each image. Reduce the collection by median (in your application, you might consider using medoid reduction to build a composite from actual data values, instead of per-band statistics).

In [20]:
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

### Display the cloud-free composite

Display the results. Be patient while the map renders, it may take a minute; [`ee.Image.reproject`](https://developers.google.com/earth-engine/guides/projections#reprojecting) is forcing computations to happen at 100 and 20 m scales (i.e. it is not relying on appropriate pyramid level [scales for analysis](https://developers.google.com/earth-engine/guides/scale#scale-of-analysis)). The issues with `ee.Image.reproject` being resource-intensive in this case are mostly confined to interactive map viewing. Batch image [exports](https://developers.google.com/earth-engine/guides/exporting) and table reduction exports where the `scale` parameter is set to typical Sentinel-2 scales (10-60 m) are less affected.



In [21]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12, height=700)

# Add layers to the folium map.
m.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

In [22]:
s2_sr_median.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B6',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min

Hopefully you now have a good sense for Sentinel-2 cloud masking in the cloud 😉 with Earth Engine.

In [23]:
# {'bucket': 'project_canopy_temp', 'resolution': 10, 'filename': '1_full_polygon_test_Nov_12_16:24:31_2020', 
#  'dest_path': 'S2_CloudFree/full_polygon_test/1_full_polygon_test_Nov_12_16:24:31_2020', 'img': <ee.image.Image object at 0x1409a65b0>, 
#  'roi': [[[9.088783, 5.753152], [9.179249, 5.703106], [9.248257, 5.792946], [9.236069, 5.845887], [9.090672, 5.844009], [9.088783, 5.753152]]], 
#  'sensor_name': 'copernicus/s2_sr'}


export = ee.batch.Export.image.toCloudStorage(
  image=s2_sr_median.select(['B4', 'B3', 'B2']),
  description="s2cloudless_test5",
  scale=10,
  region=AOI,
  fileNamePrefix='S2_CloudFree/s2cloudless_misha_polygons/',
  bucket='project-canopy-temp-2',
  maxPixels=1e13
)





In [24]:
export.start()

In [25]:
while export.active():
    print(export.status(), end="\r", flush=True)

# Sandbox

In [38]:
import folium
import geemap.eefolium as gmap

Map = gmap.Map(control=True)
Map.add_child(folium.LayerControl())

In [49]:
Full_Basin = [[[5.493164,8.276727],[5.449219,-5.703448],[31.376953,-4.959615],[31.157227,8.711359],[5.493164,8.276727]]]

AOI_full_basin = ee.Geometry.Polygon(Full_Basin)
START_DATE = '2019-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 10


vis = {'bands': 'array'}



def make_s2_sr_col(aoi=AOI_full_basin,start_date = START_DATE, end_date=END_DATE, CLOUD_FILTER=CLOUD_FILTER):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    return s2_sr_col

def visualize_geo(coords,Map = Map):
    Map = gmap.Map()
    geo_obj = ee.Geometry.Polygon(coords)
    Map.centerObject(geo_obj,3)
    Map.add_layer(geo_obj, {}, 'default display')
    Map.add_child(folium.LayerControl())
    return Map

def visualize_raster(img):
    Map = gmap.Map()
    Map.centerObject(img,10)
    Map.addLayer(img, vis)
    Map.add_child(folium.LayerControl())
    return Map
    

In [50]:
col = make_s2_sr_col()

In [51]:
img = col.first()

In [52]:
img_geo = col.first().geometry().getInfo()["coordinates"]

In [53]:
visualize_geo(coords=img_geo)

In [54]:
array = col.toArray()

In [56]:
array.arraySort

Help on method Image.arraySort in Image:

Image.arraySort(*args, **kwargs) method of ee.image.Image instance
    Sorts elements of each array pixel along one axis.
    
    Args:
      image: Array image to sort.
      keys: Optional keys to sort by. If not provided, the values are
          used as the keys. The keys can only have multiple elements
          along one axis, which determines the direction to sort in.



# Add NDVI Layer

In [24]:
# nir = img.select('B8')
# red = img.select('B4')
# ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')


def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI');
    return image.addBands(ndvi)


col2 = col.map(addNDVI)

In [30]:
ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
img = img.addBands(ndvi)

In [26]:
col2.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [32]:
NDVIpalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']

vis = {'bands': 'NDVI', 'palette':NDVIpalette}

visualize_raster(img)

In [34]:
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [36]:
img2 = img.float()

In [37]:
img2.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [5490, 5490],
   'crs': 'EPSG:32635',
   'crs_transform': [20, 0, 499980, 0, -20, 900000]},
  {'id': 'B6',
   'data_type': {'type': 'P

# Sandbox

In [189]:
import time


dest_path = 'S2_CloudFree/s2cloudless_full_tile_test/'
name = 'zhenya_full_tile_test' 

image_dl_list = col.toList(5)

for i in range(len(image_dl_list.getInfo())):
    image_to_dl = ee.Image(image_dl_list.get(i))
#     img_2 = image_to_dl.select('B.+')
    img_2 = image_to_dl.select('TCI_R', 'TCI_G', 'TCI_B')
    time_stamp = "_".join(time.ctime().split(" ")[1:])
    filename = "_".join([str(i)] + [name] + [time_stamp])
    dest_path_full = dest_path + filename
    print(dest_path_full)
    export = ee.batch.Export.image.toCloudStorage(
      image=img_2,
      description=filename,
      scale=10,
      fileNamePrefix=dest_path_full,
      bucket='project-canopy-temp-2',
      maxPixels=1e13
    )
    export.start()

S2_CloudFree/s2cloudless_full_tile_test/0_zhenya_full_tile_test_Nov_13_14:40:09_2020
S2_CloudFree/s2cloudless_full_tile_test/1_zhenya_full_tile_test_Nov_13_14:40:09_2020
S2_CloudFree/s2cloudless_full_tile_test/2_zhenya_full_tile_test_Nov_13_14:40:09_2020
S2_CloudFree/s2cloudless_full_tile_test/3_zhenya_full_tile_test_Nov_13_14:40:10_2020
S2_CloudFree/s2cloudless_full_tile_test/4_zhenya_full_tile_test_Nov_13_14:40:10_2020


In [234]:
def export_single_image(img=img,dest_path = 'S2_CloudFree/s2cloudless_mosaic_test/',name = 'zhenya_mosaic_test' ):
    
    time_stamp = "_".join(time.ctime().split(" ")[1:])
    filename = "_".join([str(0)] + [name] + [time_stamp])
    dest_path_full = dest_path + filename
    export = ee.batch.Export.image.toCloudStorage(
      image=img_2,
      description=filename,
      scale=10,
      fileNamePrefix=dest_path_full,
      bucket='project-canopy-temp-2',
      maxPixels=1e13
    )
    export.start()


# Filter by Image ID

In [248]:
def single_tile_mosaic(img=col.first(),col=col):
    ee_img_tile_id = img.get('system:index').getInfo().split("_")[-1]
    col2 = col \
        .filterMetadata('system:index', 'contains', ee_img_tile_id)
    
    print(f"mosaicing {col2.size().getInfo()} products within tile {ee_img_tile_id}")
        
    col_mosaic = col2.mosaic()
    
    return col_mosaic.select('TCI_R', 'TCI_G', 'TCI_B')


In [249]:
col_mosaic = single_tile_mosaic()

mosaicing 123 products within tile T35NNJ


In [21]:
col_mosaic.geometry().getInfo()

NameError: name 'col_mosaic' is not defined

In [243]:
export_single_image(img=col_mosaic)

# Sandbox

In [57]:
col.first().getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2A_20190101T082331_018422_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2A',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 107.260625594,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 143.345065888,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[27.996333673895258, 8.14089883917354],
   [27.996318751580215, 8.140901677298997],
   [27.890843565137047, 8.141146792902314],
   [27.89080798086886, 8.14111578493526],
   [27.890763455819915, 8.141099635632202],
   [27.890755230307672, 8.141078690077169],
   [27.89020564007191, 8.138908404504265],
   [27.888556054674908, 8.131856240006611],
   [27.885256302340263, 8.117210824008627],
   [27.78254176292224, 7.649081925146105],
   [27.680100380740086, 7.180896033870973],
   [27.673532908864704, 7.1505041677786165],
   [27.67353004444077, 7.148504745346676],
   [27.673566834382346, 7.148463192076957],
   [27.673600043788124, 7.148414

In [60]:
single_sort_nodata.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 600000, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 600000, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 600000, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 600000, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [59]:
single_sort_nodata = col.sort('NODATA_PIXEL_PERCENTAGE')
single_sort_thin_cirrus = col.sort('THIN_CIRRUS_PERCENTAGE')
double_sort = col.sort('NODATA_PIXEL_PERCENTAGE').sort('THIN_CIRRUS_PERCENTAGE')

In [198]:
single_sort_nodata_list = single_sort_nodata.aggregate_array('NODATA_PIXEL_PERCENTAGE')
single_sort_thin_cirrus_list = single_sort_thin_cirrus.aggregate_array('THIN_CIRRUS_PERCENTAGE')
doubt_sort_nodata_list = double_sort.aggregate_array('NODATA_PIXEL_PERCENTAGE')
double_sort_thin_cirrus_list = double_sort.aggregate_array('THIN_CIRRUS_PERCENTAGE')
image_id_list = double_sort.aggregate_array('system:index')

In [199]:
sort_dic = \
{"single_sort_nodata_list":single_sort_nodata_list,
 "single_sort_thin_cirrus_list":single_sort_thin_cirrus_list,
 "doubt_sort_nodata_list":doubt_sort_nodata_list,
 "double_sort_thin_cirrus_list":double_sort_thin_cirrus_list,
 "id":image_id_list}

In [200]:
new_sort_dic = {}

for key in sort_dic:
    new_sort_dic[key] = sort_dic[key].getInfo()

In [201]:
df = pd.DataFrame(new_sort_dic)

In [202]:
pd.set_option('display.max_rows', 1000)
df.head(10)

,single_sort_nodata_list,single_sort_thin_cirrus_list,doubt_sort_nodata_list,double_sort_thin_cirrus_list,id
0,0.0,0.0,0.0,0.0,20190203T083141_20190203T083954_T35NLH
1,0.0,0.0,0.0,0.0,20190206T084131_20190206T084639_T35NKJ
2,0.0,0.0,0.0,0.0,20190207T081111_20190207T082327_T36NUN
3,0.0,0.0,0.0,0.0,20190207T090139_20190207T091312_T33NYA
4,0.0,0.0,0.0,0.0,20190207T090139_20190207T091312_T33NYB
5,0.0,0.0,0.0,0.0,20190207T090139_20190207T091312_T33NYC
6,0.0,0.0,0.0,0.0,20190207T090139_20190207T091312_T33NZB
7,0.0,0.0,0.0,0.0,20190217T095051_20190217T100744_T32PKQ
8,0.0,0.0,0.0,0.0,20190218T083009_20190218T084507_T35NMG
9,0.0,0.0,0.0,0.0,20190224T084939_20190224T090717_T34NCF


In [205]:
ee.Image('COPERNICUS/S2_SR/' + df["id"].loc[0]).getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 300000, 0, -60, 800040]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 300000, 0, -10, 800040]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 300000, 0, -10, 800040]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 300000, 0, -10, 800040]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [ ]:
"20190203T083141_20190203T083954_T35NLH"

In [168]:
df["single_sort_sat_pixel_list"].value_counts()

KeyError: 'single_sort_sat_pixel_list'

In [158]:
single_sort_sat_pixel.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [454]:
col.size().getInfo()

16430

In [451]:
def image_collection_resorted(col,primary_sort='NODATA_PIXEL_PERCENTAGE',secondary_sort='CLOUDY_PIXEL_PERCENTAGE'):
    
#     img_objs = []

    new_list_of_images = []
    primary_list = col.aggregate_array(primary_sort)
    secondary_list = col.aggregate_array(secondary_sort)
    image_id_list = col.aggregate_array('system:index')
    
    
    sort_dic = \
    {primary_sort:primary_list,
     secondary_sort:secondary_list,
     "id":image_id_list}
    

    new_sort_dic = {}
    for key in sort_dic:
        new_sort_dic[key] = sort_dic[key].getInfo()
#         print(sort_dic[key])
#     return new_sort_dic
        
    df = pd.DataFrame(new_sort_dic)
    
    df = df.sort_values(by=[primary_sort,secondary_sort],ascending=False)
    
    df = df.reset_index()
    
    df = df.reset_index()
    
    df.rename(columns = {'index':'current_position', 'level_0':'new_position'}, inplace = True) 
    
    return df
    
    
    list_of_images = col.toList(col.size())
    

# for i in df.itertuples()[:3]:
#     print(i)

    for row in df.iterrows():
        origin = row[1][1]
        dest = row[1][0] 

        img_dest = ee.Image(list_of_images.get(origin))
        new_list_of_images.append(img_dest)
    
#     df = df.reset_index(drop=True)
    
    
#     for image_id in df["id"]:
#         print(image_id)
#         img = ee.Image('COPERNICUS/S2_SR/' + image_id)
#         img_objs.append(img)
        
    return ee.ImageCollection(new_list_of_images)
    

In [452]:
df = image_collection_resorted(col)

In [453]:
df

,new_position,current_position,NODATA_PIXEL_PERCENTAGE,CLOUDY_PIXEL_PERCENTAGE,id
0,0,15523,100.000000,0.0,20201019T085909_20201019T092101_T34MBC
1,1,9442,99.999905,0.0,20200124T083139_20200124T084725_T35NPH
2,2,3442,99.999887,0.0,20190426T081609_20190426T083340_T35MQV
3,3,9980,99.999875,0.0,20200203T083049_20200203T084711_T35NPH
4,4,6241,99.999815,0.0,20191008T081831_20191008T083435_T35MQV
...,...,...,...,...,...
16401,16401,14760,0.000000,0.0,20200812T084601_20200812T090428_T34MCB
16402,16402,15759,0.000000,0.0,20201105T085139_20201105T090602_T34PEQ
16403,16403,15896,0.000000,0.0,20201112T084209_20201112T085723_T34NHP
16404,16404,15904,0.000000,0.0,20201112T084209_20201112T085723_T35NKJ


In [368]:
df

,new_position,current_position,NODATA_PIXEL_PERCENTAGE,CLOUDY_PIXEL_PERCENTAGE,id
0,0,1413,0.000000,0.0,20190206T084131_20190206T084639_T35NKJ
1,1,1835,0.000000,0.0,20190218T083009_20190218T084507_T35NMG
2,2,4126,0.000000,0.0,20190608T082609_20190608T084458_T34MHV
3,3,6880,0.000000,0.0,20191130T083311_20191130T084253_T35NMH
4,4,7910,0.000000,0.0,20191227T082341_20191227T083221_T35NND
...,...,...,...,...,...
16273,16273,6241,99.999815,0.0,20191008T081831_20191008T083435_T35MQV
16274,16274,9980,99.999875,0.0,20200203T083049_20200203T084711_T35NPH
16275,16275,3442,99.999887,0.0,20190426T081609_20190426T083340_T35MQV
16276,16276,9442,99.999905,0.0,20200124T083139_20200124T084725_T35NPH


In [419]:
df

,new_position,current_position,NODATA_PIXEL_PERCENTAGE,CLOUDY_PIXEL_PERCENTAGE,id
0,0,1413,0.000000,0.0,20190206T084131_20190206T084639_T35NKJ
1,1,1835,0.000000,0.0,20190218T083009_20190218T084507_T35NMG
2,2,4126,0.000000,0.0,20190608T082609_20190608T084458_T34MHV
3,3,6880,0.000000,0.0,20191130T083311_20191130T084253_T35NMH
4,4,7910,0.000000,0.0,20191227T082341_20191227T083221_T35NND
...,...,...,...,...,...
16273,16273,6241,99.999815,0.0,20191008T081831_20191008T083435_T35MQV
16274,16274,9980,99.999875,0.0,20200203T083049_20200203T084711_T35NPH
16275,16275,3442,99.999887,0.0,20190426T081609_20190426T083340_T35MQV
16276,16276,9442,99.999905,0.0,20200124T083139_20200124T084725_T35NPH


In [376]:
dest_dict.clear()

In [409]:
t = []

t[100] = 8

IndexError: list assignment index out of range

In [429]:
list_of_images = col.toList(col.size())
# list_of_list_of_images = []
# list_of_list_of_images.append(list_of_images)
# dest_dict= {}
new_list_of_images = []

# for i in df.itertuples()[:3]:
#     print(i)

for row in df.iterrows():
    origin = row[1][1]
    dest = row[1][0] 
    
    img_dest = ee.Image(list_of_images.get(origin))
    new_list_of_images.append(img_dest)
    
#     if dest in dest_dict:
#         origin = dest_dict[dest]
    
#     if row[0] == 0:
#         print("before swap dest:",ee.Image(list_of_images.get(dest).getInfo()["properties"]['system:index']))
#         print("before swap origin:",ee.Image(list_of_images.get(origin).getInfo()["properties"]['system:index']))
#         new_list_of_images = list_of_images.swap(dest,origin)
#         dest_dict[dest] = origin 
#         list_of_list_of_images.append(new_list_of_images)
#         print("after swap dest:",ee.Image(new_list_of_images.get(dest).getInfo()["properties"]['system:index']))
#         print("after swap origin:",ee.Image(new_list_of_images.get(origin).getInfo()["properties"]['system:index']))
#         print(f"replaced {dest} with {origin}")

#     else:
#         new_list_of_images = new_list_of_images.swap(dest,origin)
#         dest_dict[dest] = origin 
#         print(f"replaced {dest} with {origin}")
#         list_of_list_of_images.append(new_list_of_images)

# new_col = ee.ImageCollection(new_list_of_images)
    

In [428]:
list(new_dict.values())[0].getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [430]:
new_list_of_images[0].getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 199980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 199980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 199980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 199980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [418]:
ee.ImageCollection(list(new_dict.values())).first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32635',
   'crs_transform': [60, 0, 499980, 0, -60, 900000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32635',
   'crs_transform': [10, 0, 499980, 0, -10, 900000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,


In [400]:
new_list_of_images.get(0).getInfo()

RecursionError: maximum recursion depth exceeded in comparison

In [398]:
new_col.first().getInfo()

RecursionError: maximum recursion depth exceeded in comparison

In [256]:
list_of_images = col.toList(col.size())

In [261]:
list_of_images.get(0) = list_of_images.get(1) 

SyntaxError: cannot assign to function call (<ipython-input-261-02c16dae46f8>, line 1)

In [269]:
list_of_images.get(0).getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2A_20190206T084131_018937_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2A',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 283.512571609,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 134.248723197,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[25.27335904995368, 8.13844990622585],
   [25.273344037218124, 8.138452660341137],
   [24.277575978814873, 8.132997823918442],
   [24.277534352060492, 8.132960994564781],
   [24.277488462866835, 8.132929598010362],
   [24.277485651473388, 8.132914747336482],
   [24.279133487601584, 7.88493398888003],
   [24.280728626781148, 7.636957436566084],
   [24.282271250981186, 7.388977760613376],
   [24.283761632838043, 7.140994989729957],
   [24.283798724543075, 7.1409537710030095],
   [24.283830374919617, 7.140908226408227],
   [24.283845247875107, 7.140905501696798],
   [25.277321030103508, 7.1456879449885395],
   [25.27736272156022, 7.145

In [270]:
list_of_images.get(1).getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2B_20190218T083009_010200_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2B',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 237.918178248,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 125.472367424,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[27.08807605724419, 6.333123982009074],
   [27.088064744996263, 6.333124689685167],
   [26.095520660236392, 6.332344270531413],
   [26.095479000999013, 6.332307637726048],
   [26.095433103692763, 6.3322764791225445],
   [26.095430308375214, 6.332261628210682],
   [26.09625997530147, 5.835765335515383],
   [26.097020748897233, 5.339267700618309],
   [26.097057538649246, 5.3392263417341015],
   [26.097088827968438, 5.339180661369294],
   [26.097103706173613, 5.339177881987811],
   [27.087909874873286, 5.3398351664635975],
   [27.087951532703244, 5.339871716425719],
   [27.087997410174395, 5.339902809833396],
   [27.088000283127236, 5.

In [272]:
new_list_of_images = list_of_images.swap(0,1)

In [274]:
new_list_of_images.get(0).getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2B_20190218T083009_010200_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2B',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 237.918178248,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 125.472367424,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[27.08807605724419, 6.333123982009074],
   [27.088064744996263, 6.333124689685167],
   [26.095520660236392, 6.332344270531413],
   [26.095479000999013, 6.332307637726048],
   [26.095433103692763, 6.3322764791225445],
   [26.095430308375214, 6.332261628210682],
   [26.09625997530147, 5.835765335515383],
   [26.097020748897233, 5.339267700618309],
   [26.097057538649246, 5.3392263417341015],
   [26.097088827968438, 5.339180661369294],
   [26.097103706173613, 5.339177881987811],
   [27.087909874873286, 5.3398351664635975],
   [27.087951532703244, 5.339871716425719],
   [27.087997410174395, 5.339902809833396],
   [27.088000283127236, 5.

In [279]:
new_col = ee.ImageCollection(new_list_of_images)

In [282]:
new_col.first().getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2B_20190218T083009_010200_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2B',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 237.918178248,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 125.472367424,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[27.08807605724419, 6.333123982009074],
   [27.088064744996263, 6.333124689685167],
   [26.095520660236392, 6.332344270531413],
   [26.095479000999013, 6.332307637726048],
   [26.095433103692763, 6.3322764791225445],
   [26.095430308375214, 6.332261628210682],
   [26.09625997530147, 5.835765335515383],
   [26.097020748897233, 5.339267700618309],
   [26.097057538649246, 5.3392263417341015],
   [26.097088827968438, 5.339180661369294],
   [26.097103706173613, 5.339177881987811],
   [27.087909874873286, 5.3398351664635975],
   [27.087951532703244, 5.339871716425719],
   [27.087997410174395, 5.339902809833396],
   [27.088000283127236, 5.

In [283]:
new_list_of_images.get(0).getInfo()["properties"]

{'DATATAKE_IDENTIFIER': 'GS2B_20190218T083009_010200_N02.11',
 'AOT_RETRIEVAL_ACCURACY': 0,
 'SPACECRAFT_NAME': 'Sentinel-2B',
 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE': 0,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 237.918178248,
 'CLOUD_SHADOW_PERCENTAGE': 0,
 'MEAN_SOLAR_AZIMUTH_ANGLE': 125.472367424,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[27.08807605724419, 6.333123982009074],
   [27.088064744996263, 6.333124689685167],
   [26.095520660236392, 6.332344270531413],
   [26.095479000999013, 6.332307637726048],
   [26.095433103692763, 6.3322764791225445],
   [26.095430308375214, 6.332261628210682],
   [26.09625997530147, 5.835765335515383],
   [26.097020748897233, 5.339267700618309],
   [26.097057538649246, 5.3392263417341015],
   [26.097088827968438, 5.339180661369294],
   [26.097103706173613, 5.339177881987811],
   [27.087909874873286, 5.3398351664635975],
   [27.087951532703244, 5.339871716425719],
   [27.087997410174395, 5.339902809833396],
   [27.088000283127236, 5.

# Attempt #2 Secondary Image Sorting

In [64]:
single_sort_nodata = col.sort('NODATA_PIXEL_PERCENTAGE')
single_sort_thin_cirrus = col.sort('THIN_CIRRUS_PERCENTAGE')
double_sort = col.sort('NODATA_PIXEL_PERCENTAGE').sort('THIN_CIRRUS_PERCENTAGE')

In [65]:
single_sort_nodata_list = single_sort_nodata.aggregate_array('NODATA_PIXEL_PERCENTAGE')
single_sort_thin_cirrus_list = single_sort_thin_cirrus.aggregate_array('THIN_CIRRUS_PERCENTAGE')
doubt_sort_nodata_list = double_sort.aggregate_array('NODATA_PIXEL_PERCENTAGE')
double_sort_thin_cirrus_list = double_sort.aggregate_array('THIN_CIRRUS_PERCENTAGE')
image_id_list = double_sort.aggregate_array('system:index')

In [66]:
sort_dic = \
{"single_sort_nodata_list":single_sort_nodata_list,
 "single_sort_thin_cirrus_list":single_sort_thin_cirrus_list,
 "doubt_sort_nodata_list":doubt_sort_nodata_list,
 "double_sort_thin_cirrus_list":double_sort_thin_cirrus_list,
 "id":image_id_list}

In [67]:
new_sort_dic = {}

for key in sort_dic:
    new_sort_dic[key] = sort_dic[key].getInfo()

In [68]:
df = pd.DataFrame(new_sort_dic)

In [77]:
pd.set_option('display.max_rows', 50)
df.tail(100)

,single_sort_nodata_list,single_sort_thin_cirrus_list,doubt_sort_nodata_list,double_sort_thin_cirrus_list,id
16648,99.881721,8.918048,95.626307,8.918048,20190319T090019_20190319T091418_T34NCH
16649,99.885792,8.921601,53.218275,8.921601,20200103T095401_20200103T100437_T32NLN
16650,99.886942,8.924355,56.051838,8.924355,20190120T094319_20190120T095914_T31NHB
16651,99.891084,8.926911,32.957649,8.926911,20191007T075809_20191007T081020_T35MRQ
16652,99.897742,8.943155,0.000000,8.943155,20200817T075621_20200817T080824_T36MUC
...,...,...,...,...,...
16743,99.999875,9.882282,0.032322,9.882282,20201119T092301_20201119T093720_T33NVH
16744,99.999887,9.887907,0.000000,9.887907,20200620T083611_20200620T085053_T34MDV
16745,99.999887,9.933465,6.053371,9.933465,20190212T090101_20190212T091820_T33MZV
16746,99.999905,9.935923,0.000000,9.935923,20190202T090201_20190202T091427_T34NBN


In [75]:
df["double_sort_thin_cirrus_list"].value_counts()

0.000000    2290
0.000003      35
0.000013      17
0.000050      16
0.000010      16
            ... 
9.490251       1
0.147316       1
8.771716       1
0.877124       1
1.777999       1
Name: double_sort_thin_cirrus_list, Length: 13347, dtype: int64

In [81]:
df[df["double_sort_thin_cirrus_list"] == 0.000010]

,single_sort_nodata_list,single_sort_thin_cirrus_list,doubt_sort_nodata_list,double_sort_thin_cirrus_list,id
2365,0.0,0.00001,0.000000,0.00001,20190224T094031_20190224T094938_T32MLE
2366,0.0,0.00001,0.000000,0.00001,20190719T084601_20190719T090348_T34MBV
2367,0.0,0.00001,0.000000,0.00001,20190816T080611_20190816T082505_T35MQU
2368,0.0,0.00001,0.000000,0.00001,20200108T095309_20200108T100226_T31NHJ
2369,0.0,0.00001,0.000000,0.00001,20201118T090249_20201118T091610_T34NBP
2370,0.0,0.00001,0.000007,0.00001,20201118T090249_20201118T091610_T34PBQ
2371,0.0,0.00001,1.060016,0.00001,20190607T085609_20190607T091155_T33MYR
2372,0.0,0.00001,1.998444,0.00001,20190613T092039_20190613T093942_T32MPV
2373,0.0,0.00001,2.460154,0.00001,20191205T092351_20191205T093444_T33NTG
2374,0.0,0.00001,3.801447,0.00001,20200201T093119_20200201T094737_T32NRN


# Using Custom Secondary Sort

In [83]:
def image_collection_secondary_sort(col,primary_sort='THIN_CIRRUS_PERCENTAGE',secondary_sort='NODATA_PIXEL_PERCENTAGE'):
    
    


    new_list_of_images = []
    primary_list = col.aggregate_array(primary_sort)
    secondary_list = col.aggregate_array(secondary_sort)
    # image_id_list = col.aggregate_array('system:index')
    
    
    # sort_dic = \
    # {primary_sort:primary_list,
    #  secondary_sort:secondary_list,
    #  "id":image_id_list}

    sort_dic = \
    {primary_sort:primary_list,
     secondary_sort:secondary_list}
    

    new_sort_dic = {}
    #print('secondary sort -- getting infos')
    for key in sort_dic:
        new_sort_dic[key] = sort_dic[key].getInfo()
        #print(f'got info for {key}')
        
    df = pd.DataFrame(new_sort_dic)
    
    df = df.sort_values(by=[primary_sort,secondary_sort],ascending=False)
    
    return df
    
    df = df.reset_index()
    
    df = df.reset_index()
    
    df.rename(columns = {'index':'current_position', 'level_0':'new_position'}, inplace = True) 
    
    
    list_of_images = col.toList(col.size())
    

    for row in df.iterrows():
        origin = row[1][1]

        img_dest = ee.Image(list_of_images.get(origin))

        new_list_of_images.append(img_dest)

        
    return ee.ImageCollection(new_list_of_images)

In [84]:
df2 = image_collection_secondary_sort(col)

In [85]:
df2

,THIN_CIRRUS_PERCENTAGE,NODATA_PIXEL_PERCENTAGE
965,9.977973,0.000003
1315,9.935923,0.000000
1621,9.933465,6.053371
13749,9.887907,0.000000
16120,9.882282,0.032322
...,...,...
16217,0.000000,0.000000
16348,0.000000,0.000000
16349,0.000000,0.000000
16502,0.000000,0.000000


In [86]:
df2[df2["THIN_CIRRUS_PERCENTAGE"] == 0.000010]

,THIN_CIRRUS_PERCENTAGE,NODATA_PIXEL_PERCENTAGE
16145,0.00001,66.784090
4148,0.00001,65.118337
16651,0.00001,35.426080
14271,0.00001,33.594659
13848,0.00001,33.470109
14400,0.00001,31.017947
9846,0.00001,3.801447
7047,0.00001,2.460154
4279,0.00001,1.998444
4098,0.00001,1.060016


# Explore Viz Parsms